In [1]:
import sys, os
sys.path.insert(0, f"C:{os.sep}Users{os.sep}xdzl45{os.sep}workspace{os.sep}molearn{os.sep}src") #uncomment/edit as appropriate for your system
from molearn.data import PDBData
from molearn.models.foldingnet import AutoEncoder
from molearn.analysis import MolearnAnalysis, MolearnGUI
import torch
from copy import deepcopy
import biobox as bb
import numpy as np


# input files
networkfile = f'data{os.sep}checkpoint_no_optimizer_state_dict_epoch167_loss0.003259085263643.ckpt'
training_set_files = [f'data{os.sep}MurD_open_selection.pdb', f'data{os.sep}MurD_open_selection.pdb']
test_set_file = f'data{os.sep}MurD_closed_apo_selection.pdb'

print("Loading network")
checkpoint = torch.load(networkfile, map_location= torch.device('cpu'))
net = AutoEncoder(**checkpoint['network_kwargs'])
net.load_state_dict(checkpoint['model_state_dict'])

print("Loading training set")
data = PDBData()
for f in training_set_files:
    data.import_pdb(f)
data.atomselect(atoms=['CA', 'C', 'CB', 'N', 'O'])
data.prepare_dataset()
train_data, valid_data = data.split(manual_seed=25) # same manual seed used in the training script

print("Loading test set")
test_data = PDBData()
test_data.import_pdb(test_set_file)
test_data.atomselect(atoms=['CA', 'C', 'CB', 'N', 'O'])
test_data.std = data.std
test_data.prepare_dataset()

MA = MolearnAnalysis()
MA.set_network(net)
MA.set_dataset('training', train_data)
MA.set_dataset('validation', valid_data)
MA.set_dataset('test', test_data)

#print("Scanning landscape error")
#MA.setup_grid(30)
#landscape_err_latent, landscape_err_3d, xaxis, yaxis = MA.scan_error()

#print("Scanning landscape with custom function")
#def f_distance(crd, M, idx1, idx2):
#    M.coordinates = deepcopy(crd)
#    M.set_current(0)
#    return np.sqrt(np.sum(M.points[idx2]-M.points[idx1])**2)

#_, idx1 = MA.mol.atomselect("*", 1, "CA", get_index=True) # find index of first atom of interest
#_, idx2 = MA.mol.atomselect("*", 300, "CA", get_index=True)  # find index of second atom of interest
#dist, xaxis, yaxis = MA.scan_custom(f_distance, [MA.mol, idx1, idx2], "dist", samples=50)

C:\Users\xdzl45\workspace\molearn\src\molearn\loss_functions\openmm_thread.py:13: UserWarning: No module named 'torchexposedintegratorplugin'. Will not be able to use openmm.
  warnings.warn(f'{e}. Will not be able to use openmm.')
C:\Users\xdzl45\workspace\molearn\src\molearn\trainers\sinkhorn_trainer.py:15: UserWarning: No module named 'geomloss'. Will not be able to use sinkhorn because geomloss is not installed.
  warnings.warn(f'{e}. Will not be able to use sinkhorn because geomloss is not installed.')


Loading network
Loading data
Dataset.shape: torch.Size([1800, 3, 2145])
mean: 44.57135474712939, std: 14.056555172534024
Dataset.shape: torch.Size([900, 3, 2145])
mean: 44.04523070482608, std: 14.056555172534024


In [ ]:
MG = MolearnGUI(MA)

---